<a href="https://colab.research.google.com/github/mildinvestor/katago-colab/blob/master/colab_katago_gd_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document shows how to run KataGo on Colab, and how to connect it using `Sabaki`, `Lizzie` (or other GTP engine supported apps) in your local machine.

# Step 1: Run KataGo and SSH server on Colab Server

Before running, you should prepare these 2 values: `NGROK_TOKEN` and `USER_PASSWORD`. 

* NGROK_TOKEN - You can register or login in `ngrok` website here: [https://dashboard.ngrok.com/auth/your-authtoken](https://dashboard.ngrok.com/auth/your-authtoken). Then you can find your own ngrok token.
* USER_PASSWORD - You can use any password as you wish. (should only contains letters or digits)

`USER_PASSWORD` will be used in `Sabaki` or `Lizzie` as engine options. (More details will be elaborated in Step 2 below).

Change these 2 values of your own in the parameter from the right parameter entry field(`Config for SSH`). 

In addition, change the `Config for KataGo` as needed.

* WEIGHT_FILE - You can choose the number of blocks in your deep learning network file. If you set it to "AUTO", it will be automatically set according to the performance of the GPU assigned to Google Colab.
* rules - You can choose the rules of Go.
* maxVisits - You can enter the maximum number of visits to the node. The larger this number, the more time KataGo will take to search the game tree during a game(in genmove mode).

Then click the `Run` button. 

After run, do the following:
* Authenticate your Google Drive. Please access the authentication URL displayed, log in to Google, and then allow authentication. Enter the verification code displayed in the `Enter verification code:` field.
* Copy the value of `SSH_INFO_GOOGLE_DRIVE_FILE_ID` displayed in the output cell. This will be used as engine options. This value will be the same after the next execution.

It may take about 2~3 minutes to complete running the code. 

In [ ]:
#@markdown SSH configuration
NGROK_TOKEN = 'Input your ngrok token' #@param {type:"string"}
USER_PASSWORD = 'InputYourPassword' #@param {type:"string"}

#@markdown KataGo configuration
WEIGHT_FILE = 'AUTO' #@param ["AUTO", "20b", "40b"]
rules = 'japanese' #@param ["tromp-taylor", "chinese", "japanese", "korean", "aga", "chinese-ogs", "new-zealand"]
maxVisits = 5000 #@param {type:"integer"}

# Constant
KATAGO_COLAB_REPOSITORY_URL = 'https://github.com/mildinvestor/katago-colab.git'
KATAGO_CONFIG_FILE = '/content/katago-colab/config/gtp_colab.cfg'
KATAGO_TUNING_DIR = '/content/katago-colab/opencltuning'
KATAGO_RESOURCE_FILE = '/content/katago-colab/colab-resource/external-resource.json'
SSH_INFO_FILE_NAME = 'colab-katago-ssh.json'

# Install useful stuff
!echo "Install libraries"
!apt-get update 1>/dev/null
!apt-get install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!pip install -U -q PyDrive 1>/dev/null

import subprocess
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
import json
from re import sub

def get_katago_resource(katago_resource_file):
    f = open(katago_resource_file, 'r')
    try:
        resource = json.load(f)
    finally:
        f.close()

# If you want to change the file to the one you prefer, change the "resource" variable as shown in the example below.
#    resource = {
#        "katago": {
#            "bin": "https://github.com/mildinvestor/katago-colab/releases/download/v1.9.1-alpha/katago",
#            "weights": {
#                "40b": "https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s9854456576-d2405111631.bin.gz",
#                "20b": "https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b20c256x2-s5303129600-d1228401921.bin.gz"
#            }
#        },
#        "ngrok": "https://github.com/mildinvestor/katago-colab/releases/download/v1.9.1-alpha/ngrok"
#    }
    return resource


def get_weight_type(weight, gpu_name):
    if weight == 'AUTO':
        if gpu_name == 'TeslaK80' or gpu_name == 'TeslaP4':
            return '20b'
        else:
            return '40b'
    else:
        return weight


def auth_google_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive


def save_ssh_info(file_name, drive):
    # Get ssh option
    r = requests.get('http://localhost:4040/api/tunnels')
    raw_ssh = r.json()['tunnels'][0]['public_url']
    ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
    ssh_option = {
        'host': ssh_args[0],
        'port': int(ssh_args[1]),
        'user': 'root'
    }

    # Save ssh info to Google Drive
    file_metadata = {
        'title': file_name,
        'mimeType': 'application/json'
    }

    file_list = drive.ListFile({'q': 'title="' + file_name + '" and trashed=False'}).GetList()
    if (len(file_list) > 0):
        file_metadata['id'] = file_list[0]['id']

    ssh_option_file = drive.CreateFile(file_metadata)
    ssh_option_file.SetContentString(json.dumps(ssh_option))
    ssh_option_file.Upload()
    ssh_option_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})

    print('===========================================================================')
    print('Please copy the following parameter that will be used in Sabaki or Lizzie as engine options.')
    print('SSH_INFO_GOOGLE_DRIVE_FILE_ID: {}'.format(ssh_option_file.get('id')))
    print('===========================================================================')


def overwrite_katago_config(config_file, rules, max_visits):
    # Overwrite KataGo config
    with open(config_file, mode='w') as f:
        f.write('logDir = gtp_logs\n')
        f.write('logAllGTPCommunication = true\n')
        f.write('logSearchInfo = true\n')
        f.write('logToStderr = false\n')
        f.write('rules = ' + rules + '\n')
        f.write('allowResignation = true\n')
        f.write('resignThreshold = -0.90\n')
        f.write('resignConsecTurns = 3\n')
        if max_visits > 0:
            f.write('maxVisits = ' + str(max_visits) +'\n')
        f.write('ponderingEnabled = true\n')
        f.write('maxTimePondering = 60\n')
        f.write('lagBuffer = 1.0\n')
        f.write('numSearchThreads = 12\n')
        f.write('searchFactorAfterOnePass = 0.50\n')
        f.write('searchFactorAfterTwoPass = 0.25\n')
        f.write('searchFactorWhenWinning = 0.40\n')
        f.write('searchFactorWhenWinningThreshold = 0.95\n')


# Get GPU name
gpu_name = str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
print('GPU: {}'.format(gpu_name))

# Select weight file
WEIGHT_FILE = get_weight_type(WEIGHT_FILE, gpu_name)
print('Using KataGo Weight: {}'.format(WEIGHT_FILE))

# Authenticate Google Drive
!echo "Authenticate Google Drive"
drive = auth_google_drive()

%cd /content
# Clone katago-colab
!echo "Git clone katago-colab"
!rm -rf katago-colab
!git clone $KATAGO_COLAB_REPOSITORY_URL 1>/dev/null

# Get URLs of external resources
katago_resource = get_katago_resource(KATAGO_RESOURCE_FILE)
katago_url = katago_resource['katago']['bin']
ngrok_url = katago_resource['ngrok']
weight_urls = katago_resource['katago']['weights']

# Download ngrok
!wget --quiet $ngrok_url -O ngrok
!chmod +x /content/ngrok
 
# SSH setting
!echo "root:$USER_PASSWORD" | chpasswd
!echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
!echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

!mkdir -p /root/.ssh
!service ssh restart > /dev/null
 
# Run ngrok
!echo "Run ngrok"
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
!sleep 5

# Save SSH info to Google Drive
!echo "Save SSH info to Google Drive"
save_ssh_info(SSH_INFO_FILE_NAME, drive)

# Download KataGo binary
!echo "Install KataGo"
!wget --quiet $katago_url -O katago
!chmod +x /content/katago
!/content/katago version

# Put KataGo tuning files
!mkdir -p /root/.katago/
!cp -r $KATAGO_TUNING_DIR /root/.katago/

# Download a network file of KataGo
weight_url = weight_urls[WEIGHT_FILE]
!wget --quiet $weight_url -O $WEIGHT_FILE".bin.gz" 
!rm -rf weight.bin.gz
!ln -s $WEIGHT_FILE".bin.gz" weight.bin.gz

# Overwrite katago config
overwrite_katago_config(KATAGO_CONFIG_FILE, rules, maxVisits)

!echo -e "\n[KataGo Config]" 
!cat $KATAGO_CONFIG_FILE
!echo -e "\n"

!echo "done!"

#Step 2: Connect Colab KataGo via Sabaki or Lizzie

## 1) first, download a colab-katago client app
Here are the download Links:  

**For Windows Users (64bit windows)**  
https://github.com/mildinvestor/katago-colab/releases/download/v1.9.1-alpha/colab-katago-gd.windows.zip

**For Linux Users**  
https://github.com/mildinvestor/katago-colab/releases/download/v1.9.1-alpha/colab-katago-gd.linux.zip

**For Mac OSX Users**  
https://github.com/mildinvestor/katago-colab/releases/download/v1.9.1-alpha/colab-katago-gd.mac.zip

**after download completed, unzip it, you can get a binary program naming  colab-katago-gd or colab-katago-gd.exe**

## 2) then, add engine in Sabaki or Lizzie

To configure the engine in `Sabaki` or `Lizzie`, you just need to fill the absolute path of your `colab-katago` program (which you've downloaded just now), and the file ID, password(i.e, `SSH_INFO_GOOGLE_DRIVE_FILE_ID` displayed in the output cell, `USER_PASSWORD` you configured in Step 1).

```
<Absolute path of colab-katago-gd> <SSH_INFO_GOOGLE_DRIVE_FILE_ID> <USER_PASSWORD>
```

**Sabaki Example**:  

![Sabaki Example Image](
https://mildinvestor.com/wp-content/uploads/2021/08/colab-katago-sabaki-example.jpg
)  

**Lizzie Example**:  

![Lizzie Example Image](https://mildinvestor.com/wp-content/uploads/2021/08/colab-katago-lizzie-example.jpg
)

## More Config Options (Optional)
You can use the following engine options to limits the KataGo Search `visits` or `time` (in seconds) in genmove mode. For example: 

```
<AbsolutePathOfColabKataGoProglem> <SSH_INFO_GOOGLE_DRIVE_FILE_ID> <USER_PASSWORD> 30s
```
The above options `30s` limits the search time for each move to 30 seconds.
```
<AbsolutePathOfColabKataGoProglem> <SSH_INFO_GOOGLE_DRIVE_FILE_ID> <USER_PASSWORD> 1600v
```
The above options `1600v` limits the search visits for each move to 1600 visits.

You can change the numbers 30 or 1600 in the above to any number as you want.

#The following sections are for debugging only, you can ignore.

**Shows the Colab GPU Info**

In [ ]:
!nvidia-smi

Get your ssh login info
*ssh login account is root, the login password is the `USER_PASSWORD` you configured in the previous steps*

In [ ]:
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh 
print(str_ssh)

**Restart SSH**


In [ ]:
!service ssh restart